# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
from skimage import io
from skimage import img_as_uint
from glob import glob
import numpy as np
import tifffile
from denoiser import Denoiser
from lsm_utils import normalize_16bit_images, compute_norm_range
import yaml

# Load data 
- SHG samples
- load model config data from yaml file

In [4]:
def sandbox_compute_norm_range(fname, percentiles=(0, 100), sample_r=0.1):
    max_val = []
    min_val = []
    fail_names = []
    try:
        img = img_as_uint(io.imread(fname))
    except Exception as e:
        print(e)
        print(fname)
        fail_names.append(fname)
    max_val.append(np.percentile(img, percentiles[1]))
    min_val.append(np.percentile(img, percentiles[0]))
    max_val: float | np.ndarray = np.percentile(np.array(max_val), 98)
    min_val: float | np.ndarray = np.percentile(np.array(min_val), 2)
    
    return min_val, max_val, fail_names

# vmin, vmax, fail_names = sandbox_compute_norm_range('sample_data/1B_C1.tif', percentiles=(1, 99.5), sample_r=0.05)
vmin, vmax, fail_names = compute_norm_range('sample_data', ext='tif', percentiles=(1, 99.5), sample_r=1)

sample_data\*.tif


100%|██████████| 1/1 [00:00<00:00,  2.40it/s]


In [5]:
config = yaml.load(open("model_config.yaml", "r"), Loader=yaml.FullLoader)
config['dataset'] = 'sample_data'
config['norm-range'] = [int(vmin), int(vmax)]
config['threads'] = 0

# Create model instance
* Create instance of denoiser with new config data

In [8]:
# Set background screening to false to allow PB522-14-MAX-Fused.tif to be accepted
denoiser = Denoiser(config, screen_bg=False)

Using cache found in C:\Users\aarus/.cache\torch\hub\mateuszbuda_brain-segmentation-pytorch_master


In [9]:
# Denoiser only works on grayscale images
denoiser.denoise()

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.39 GiB. GPU 0 has a total capacty of 6.00 GiB of which 0 bytes is free. Of the allocated memory 10.78 GiB is allocated by PyTorch, and 105.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF